# Classification:

While Linear Regression is used to predict a numeric value, classification is used to separate data into different labels. 

In this example, I will use the TensorFlow estimator to classify flowers. 

Sourced mainly from: https://www.tensorflow.org/tutorials/estimator/premade

Note that there won't be as many notes, as most of it is covered in previous lessons (and are therefore in the previous Jupyter Notebook)

# Imports and Setup:

In [34]:
#importing various libraries into this session:

from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd



# Data Set:
This specific dataset will seperate flowers into 3 different classes/species:
* Setosa
* Versicolor
* Virginica

Information is given for each flower, this includes:
* Sepal Length
* Sepal Width
* Petal Length
* Petal Width

In [35]:
#Creating some constants:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [36]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)


#Here we are using Keras (a module from within TensorFlow) to grab the datasets and read them into a pandas dataframe.

Time to have a look at the data:

In [37]:
train.head() #First 5 lines of the dataframe

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


For each of the datasets, split out the labels, which the model will be trained to predict.

In [38]:
train_y = train.pop('Species')
test_y = test.pop('Species')

# The label column has now been removed from the features.
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


Looking at the shape to make sure it has the expected amount of entries:

In [39]:
train.shape # Expected to have 120 entries (i.e. rows) + 4 features (i.e. columns)

(120, 4)

# Input Function:

This input function is a little easier to understand compared to the one used in the Linear Regression example.

In [40]:
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)

# Feature Columns:

Here I'll put in the feature columns.

In [41]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


# Building the model:
Now you are able to choose a model. For Classification, there are a few estimators/models that you can pick from, some examples are:
* ``tf.estimator.DNNClassifier`` for deep models that perform multi-class classification.
* ``tf.estimator.DNNLinearCombinedClassifier`` for wide & deep models.
* ``tf.estimator.LinearClassifier`` for classifiers based on linear models.

You are able to choose either from a deep neural network (DDN) or from a Linear model. As there is unlikely to be a linear relationship, it makes sense to use the deep neural network for this use-case.

In [42]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers (middle layers) of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp91ahzimm', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Training the Model
Now it is finally time to start training this Machine Learning model!

**Lambda Function**:
The lambda function allows for the creation of a function within one line.

Here is an example:

In [43]:
x = lambda: print('this is a test')

x()

this is a test


In [45]:
# Train the Model.
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True), #lambda means a one line function: everything afterwards works as a function
    steps=5000) #similar to an epoch, although it goes through 5000 iterations.

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp91ahzimm/model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2022-01-18 16:27:17.335333: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-18 16:27:17.335781: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-18 16:27:17.336136: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-18 16:27:17.336581: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-18 16:27:17.336935: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 5000...
INFO:tensorflow:Saving checkpoints for 5000 into /tmp/tmp91ahzimm/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 5000...
INFO:tensorflow:loss = 0.5976476, step = 5000
INFO:tensorflow:global_step/sec: 191.176
INFO:tensorflow:loss = 0.587281, step = 5100 (0.524 sec)
INFO:tensorflow:global_step/sec: 208.233
INFO:tensorflow:loss = 0.57784796, step = 5200 (0.480 sec)
INFO:tensorflow:global_step/sec: 242.566
INFO:tensorflow:loss = 0.58259004, step = 5300 (0.412 sec)
INFO:tensorflow:global_step/sec: 240.612
INFO:tensorflow:loss = 0.56883466, step = 5400 (0.416 sec)
INFO:tensorflow:global_step/sec: 266.583
INFO:tensorflow:loss = 0.57379997, step = 5500 (0.375 sec)
INFO:tensorflow:global_step/sec: 210.122
INFO:tensorflow:loss = 0.570787, step = 5600 (0.476 sec)
INFO:tensorflow:global_step/sec: 230.697
INFO:tensorflow:loss = 0.5670659, step = 5700 (0.433 sec)
INFO:tensorflow:global_ste

In [46]:
classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-01-18T16:28:28
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp91ahzimm/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2022-01-18 16:28:28.803227: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-18 16:28:28.803695: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-18 16:28:28.803998: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-18 16:28:28.804400: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-18 16:28:28.804708: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

KeyboardInterrupt: 